In [1]:
try:
    import pylibjpeg
except:
    !rm -rf /root/.cache/torch/hub/checkpoints/
    !mkdir -p /root/.cache/torch/hub/checkpoints/
    !pip install /kaggle/input/for-pydicom/dicomsdl-0.109.1-cp310-cp310-manylinux_2_12_x86_64.manylinux2010_x86_64.whl
    !pip install /kaggle/input/for-pydicom/python_gdcm-3.0.22-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
    !pip install /kaggle/input/for-pydicom/pylibjpeg_libjpeg-1.3.4-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
    !pip install /kaggle/input/for-pydicom/pylibjpeg-1.4.0-py3-none-any.whl

#!pip -q install dicomsdl
#!pip install pylibjpeg
#!pip install python-gdcm

#!pip install /kaggle/input/kaggle-rsna-pkgs/pylibjpeg-1.4.0-py3-none-any.whl
#!pip install /kaggle/input/kaggle-rsna-pkgs/python_gdcm-3.0.21-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
#!pip install /kaggle/input/kaggle-rsna-pkgs/pydicom-2.3.1-py3-none-any.whl

#for dirname, _, filenames in os.walk('/kaggle/input'):
#    for filename in filenames:
#        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

Processing /kaggle/input/for-pydicom/dicomsdl-0.109.1-cp310-cp310-manylinux_2_12_x86_64.manylinux2010_x86_64.whl
Processing /kaggle/input/for-pydicom/python_gdcm-3.0.22-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
Processing /kaggle/input/for-pydicom/pylibjpeg_libjpeg-1.3.4-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
Processing /kaggle/input/for-pydicom/pylibjpeg-1.4.0-py3-none-any.whl


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import pylibjpeg
import cv2
import os
import gdcm
import glob
import seaborn as sns
import pydicom
from joblib import Parallel, delayed
from tqdm.notebook import tqdm
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import Layer

/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZTVN10tenso

In [3]:
DEBUG=False

In [4]:
path = '/kaggle/input/rsna-breast-cancer-detection/'
test_csv = path + 'test.csv'
test_path = path + 'test_images/'
sample_submission = path + 'sample_submission.csv'
output_path = '/kaggle/working/'
test_output_path = output_path + 'test_images/'

img_size =256

In [5]:
test_df = pd.read_csv(test_csv)

In [6]:
def dicom_to_png(dicom_path):
    dicom = pydicom.dcmread(dicom_path)
    img = dicom.pixel_array
    img = (img - img.min()) / (img.max() - img.min())
    
    if dicom.PhotometricInterpretation == 'MONOCHROME1':
        img = 1 - img

    img = cv2.resize(img, (img_size, img_size), interpolation=cv2.INTER_LINEAR)
    img = (img * 255).astype(np.uint8)
    return img

def process(dicom_path, image_path):
    image = dicom_to_png(dicom_path)
    os.makedirs(os.path.dirname(image_path), exist_ok=True)
    cv2.imwrite(image_path, image)

In [7]:
test_df['dicom_path'] = test_path + test_df['patient_id'].astype(str) + '/' + test_df['image_id'].astype(str) + '.dcm'
test_df['image_path'] = test_output_path + test_df['patient_id'].astype(str) + '/' + test_df['image_id'].astype(str) + '.png'
test_df

,site_id,patient_id,image_id,laterality,view,age,implant,machine_id,prediction_id,dicom_path,image_path
0,2,10008,736471439,L,MLO,81,0,21,10008_L,/kaggle/input/rsna-breast-cancer-detection/tes...,/kaggle/working/test_images/10008/736471439.png
1,2,10008,1591370361,L,CC,81,0,21,10008_L,/kaggle/input/rsna-breast-cancer-detection/tes...,/kaggle/working/test_images/10008/1591370361.png
2,2,10008,68070693,R,MLO,81,0,21,10008_R,/kaggle/input/rsna-breast-cancer-detection/tes...,/kaggle/working/test_images/10008/68070693.png
3,2,10008,361203119,R,CC,81,0,21,10008_R,/kaggle/input/rsna-breast-cancer-detection/tes...,/kaggle/working/test_images/10008/361203119.png


In [8]:
Parallel(n_jobs=4, backend='threading')(delayed(process)(dicom_path, image_path) 
                   for dicom_path, image_path in tqdm(zip(test_df['dicom_path'], 
                                                          test_df['image_path'])))

0it [00:00, ?it/s]

[None, None, None, None]

In [9]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.utils import class_weight
from tensorflow.keras.models import load_model

def predict_model(model_name, generator):
    model = load_model(model_name)
    pred = model.predict(generator)
    
    y_pred = []
    for prob in pred:
        if prob >= 0.5:
            y_pred.append(1)
        else:
            y_pred.append(0)
    
    y_true = generator.classes
    
    return (y_pred, y_true, pred)

In [10]:
import tensorflow as tf
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator

test_datagen = ImageDataGenerator()
test_generator = test_datagen.flow_from_directory(
    #dir_path_test,
    "/kaggle/working/test_images/",
    target_size = (img_size, img_size),
    batch_size = 64,
    class_mode = 'binary')

Found 4 images belonging to 1 classes.


In [11]:
def pfbeta(labels, predictions, beta):
    y_true_count = 0
    ctp = 0
    cfp = 0

    for idx in range(len(labels)):
        prediction = min(max(predictions[idx], 0), 1)
        if (labels[idx]):
            y_true_count += 1
            ctp += prediction
        else:
            cfp += prediction

    beta_squared = beta * beta
    c_precision = ctp / (ctp + cfp)
    c_recall = ctp / y_true_count
    if (c_precision > 0 and c_recall > 0):
        result = (1 + beta_squared) * (c_precision * c_recall) / (beta_squared * c_precision + c_recall)
        return result
    else:
        return 0

In [12]:
test_df['cancer'] = 0
test_df['prediction_id'] = test_df['patient_id'].astype(str) + '_' + test_df['laterality']
model = load_model('/kaggle/input/baseline/baseline_vgg16.h5')
pred = model.predict(test_generator)

1/1 [==============================] - 6s 6s/step


In [13]:
pf = pfbeta(test_generator, pred, 1)
pf

array([0.56671613], dtype=float32)

In [14]:
test_df["cancer"] = pred
test_df

,site_id,patient_id,image_id,laterality,view,age,implant,machine_id,prediction_id,dicom_path,image_path,cancer
0,2,10008,736471439,L,MLO,81,0,21,10008_L,/kaggle/input/rsna-breast-cancer-detection/tes...,/kaggle/working/test_images/10008/736471439.png,0.395397
1,2,10008,1591370361,L,CC,81,0,21,10008_L,/kaggle/input/rsna-breast-cancer-detection/tes...,/kaggle/working/test_images/10008/1591370361.png,0.378988
2,2,10008,68070693,R,MLO,81,0,21,10008_R,/kaggle/input/rsna-breast-cancer-detection/tes...,/kaggle/working/test_images/10008/68070693.png,0.253058
3,2,10008,361203119,R,CC,81,0,21,10008_R,/kaggle/input/rsna-breast-cancer-detection/tes...,/kaggle/working/test_images/10008/361203119.png,0.181860


In [15]:
pre_result = test_df[['prediction_id', 'cancer']].groupby("prediction_id").agg(lambda x: pfbeta(test_generator, x.array, 1)).reset_index()
pre_result

,prediction_id,cancer
0,10008_L,0.566716
1,10008_R,0.403905


In [16]:
has_cancer = (pre_result['cancer'] > 0.3).astype(float)
pre_result['cancer'] = has_cancer
pre_result.to_csv('/kaggle/working/submission.csv', index=False)
pre_result

,prediction_id,cancer
0,10008_L,1.0
1,10008_R,1.0


In [17]:
if DEBUG:
    cm = confusion_matrix(y_true, y_pred)
    cm_pct = confusion_matrix(y_true, y_pred, normalize = "true")

    class_names = ['Negativo', 'Positivo']

    ax = sns.heatmap(cm_pct*100, annot = True, fmt = '.2f', cmap = "Blues", annot_kws = {"size": 16}, xticklabels = class_names, yticklabels = class_names)

    i=0
    for t in ax.texts:
      t.set_text(str(cm.flatten()[i])+ "\n("+ t.get_text() + "%)")
      i = i+1

    ax.set_xlabel("Predição")
    ax.set_ylabel("Real")